In [1]:
import json
import tkp.db
import tkp.config
import logging
import csv

import numpy as np
import pandas as pd
import scipy as sp
import healpy as hp
import datetime
import os
import glob
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, match_coordinates_sky,  AltAz, EarthLocation
from astropy.io import fits
from astropy.wcs import WCS


RuntimeError: module compiled against API version 0xc but this version of numpy is 0xa

/afhome/kuiack/newtrap/local/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
def run_query(transients_query, dbname, dataset_id, engine, host, port, user, pword):
    tkp.db.Database(
        database=dbname, user=user, password=pword,
        engine=engine, host=host, port=port
    )

    cursor = tkp.db.execute(transients_query, (dataset_id,))
    transients = tkp.db.generic.get_db_rows_as_dicts(cursor)
    print "Found", len(transients), "new sources"
    return transients

In [3]:
engine = 'postgresql'
host = 'ads001'
port = 5432
user = 'mkuiack'
password = 'Morecomplicatedpass1234'

query_loglevel = logging.WARNING



In [30]:
survey_stats = pd.read_csv("~/survey_stats.csv")

CS002 = EarthLocation.from_geocentric (3826577.109500000, 461022.900196000, 5064892.758, 'm')
position = SkyCoord(148.59*u.degree, 7.68*u.degree)


In [ ]:
for line in range(len(survey_stats)):
    print survey_stats.loc[line].obs, survey_stats.loc[line].dataset

    dbname = survey_stats.loc[line].obs
    dataset = survey_stats.loc[line].dataset

    logging.getLogger('sqlalchemy.engine').setLevel(query_loglevel)

    db = tkp.db.Database(engine=engine, host=host, port=port,
                     user=user, password=password, database=dbname)

    db.connect()
    session = db.Session()

    transients_query = """
        SELECT taustart_ts, band FROM image ORDER BY taustart_ts
        """
    times = pd.DataFrame(run_query(transients_query, dbname, dataset, engine, host, port, user, password))

    db._configured = False
    del db, session


    altaz = position.transform_to(AltAz(obstime=[i for i in pd.to_datetime(np.unique(times.taustart_ts.dt.round("1s")))],
                                    location=CS002))
    count = np.sum([(90.-altaz.alt.deg < 50.)])


    starttime = datetime.datetime.strftime(times.iloc[0].values[1], format="%Y-%m-%dT%H:%M:%S")
    endtime = datetime.datetime.strftime(times.iloc[-1].values[1], format="%Y-%m-%dT%H:%M:%S")
    print dbname, count, starttime, endtime
    fields=[dbname, count, starttime, endtime]
    with open(r'/home/kuiack/PSR_0950_uptime.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)

In [6]:
survey_stats

,obs,dataset,timestamp,candidate,pass
0,_201608311510,1,9227,351,8
1,_201609051647,1,10726,597,11
2,_201609070340,1,21352,364,22
3,_201611120632,1,28203,686,43
4,_201611132000,1,10316,302,14
5,_201611140501,1,13977,370,19
6,_201702241630,1,6489,151,6
7,_201702250130,2,13071,913,44
8,_201702250800,1,21426,1345,70
9,_201702251405,1,14860,900,51


In [31]:
line = 13

print survey_stats.loc[line].obs, survey_stats.loc[line].dataset

dbname = survey_stats.loc[line].obs
dataset = survey_stats.loc[line].dataset

logging.getLogger('sqlalchemy.engine').setLevel(query_loglevel)

db = tkp.db.Database(engine=engine, host=host, port=port,
                 user=user, password=password, database=dbname)

db.connect()
session = db.Session()

transients_query = """
    SELECT taustart_ts, band FROM image ORDER BY taustart_ts
    """
times = pd.DataFrame(run_query(transients_query, dbname, dataset, engine, host, port, user, password))

db._configured = False
del db, session


altaz = position.transform_to(AltAz(obstime=[i for i in pd.to_datetime(np.unique(times.taustart_ts.dt.round("1s")))],
                                location=CS002))
count = np.sum([(90. - altaz.alt.deg < 50.)])


starttime = datetime.datetime.strftime(times.iloc[0].values[1], format="%Y-%m-%dT%H:%M:%S")
endtime = datetime.datetime.strftime(times.iloc[-1].values[1], format="%Y-%m-%dT%H:%M:%S")
print dbname, count, starttime, endtime
fields=[dbname, count, starttime, endtime]
# with open(r'/home/kuiack/PSR_0950_uptime.csv', 'a') as f:
#     writer = csv.writer(f)
#     writer.writerow(fields)

_201702270350 1
Found 8798 new sources
_201702270350 1 2017-02-13T12:41:37 2017-07-13T12:48:27


In [11]:
np.array([i for i in pd.to_datetime(np.unique(times.taustart_ts.dt.round("1s")))])[-1]

Timestamp('2017-07-13 12:48:27')

False

In [8]:
altaz.alt.deg[-1]

42.310453400356757

In [10]:

print altaz.az.deg[-1], altaz.alt.deg[-1], 

155.195993654 42.3104534004


In [26]:
CS002 = EarthLocation.from_geocentric (3826577.109500000, 461022.900196000, 5064892.758, 'm')
position = SkyCoord([6.7,253,329.37]*u.degree, [64.15,4.6,37.83]*u.degree)

altaz = position.transform_to(AltAz(obstime=Time("2019-02-28T05:57:42", scale="utc"), location=CS002))

print altaz.az, altaz.alt


[30d32m58.781s 181d06m28.6722s 74d25m00.0345s] [38d04m22.1501s 41d38m52.4284s 37d32m32.9115s]


In [21]:
AltAz(obstime=Time("2019-02-28T05:57:42", scale="utc"))

<AltAz Frame (obstime=2019-02-28T05:57:42.000, location=None, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron)>